Move the cells I used to calculate the tides from "Transport" so just read them in there

In [5]:
import arrow
import datetime
import matplotlib.pyplot as plt
import os
import xarray as xr

%matplotlib inline

In [6]:
basedir = '/results2/SalishSea/nowcast-green.201806/'
iY = 344; iX = 276; iZ = 10 

In [ ]:
year = 2017
start = datetime.datetime(year, 1, 1)
endtime = datetime.datetime(year, 12, 31)
timerange = arrow.Arrow.range('day', start, endtime)
for i, day in enumerate(timerange):
    dir1 = day.format('DDMMMYY').lower()
    ymd = day.format('YYYYMMDD')
    filename = 'SalishSea_1h_'+ymd+'_'+ymd+'_grid_U.nc'
    fullfile = os.path.join(basedir, dir1, filename)
    u_vel = xr.open_dataset(fullfile)
    velocity = u_vel['vozocrtx'].isel(y=iY, x=iX).sel(depthu=iZ, method='nearest')
    u_vel.close()
    if i == 0:
        velocity_year = velocity.copy(deep=True)
        velocity.close()
    else:
        velocity_year = xr.concat([velocity_year, velocity], dim='time_counter')
        velocity.close()
    if i % 10 == 0:
        print (i)

0


In [ ]:
2+2

In [ ]:
velocity_year.plot()

In [ ]:
velocity_year.to_netcdf('velocity_2016.nc')

In [ ]:
u_vel   = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DuGridFields1hV17-02')
iY = 344; iX = 276; iZ = 10 
velocity2015 = u_vel.uVelocity.sel(time ='2015', depth=iZ, gridX=iX, gridY=iY, method='nearest')
velocity2016 = u_vel.uVelocity.sel(time ='2016', depth=iZ, gridX=iX, gridY=iY, method='nearest')
velocity2017 = u_vel.uVelocity.sel(time ='2017', depth=iZ, gridX=iX, gridY=iY, method='nearest')

In [ ]:
# Yes, do run this.  Its gets the data which makes the next cell happy.
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
velocity2015.plot()
velocity2016.plot()
velocity2017.plot()

In [ ]:
velocity = xr.concat([velocity2015, velocity2016], dim='time')
velsquared = velocity * velocity
day_avg_tide_vel = velsquared.resample('1D', dim='time', how='mean')

In [ ]:
day_avg_tide_pd = day_avg_tide_vel.to_dataframe()
day_avg_tide_pd = day_avg_tide_pd.drop('depth', 1)
day_avg_tide_pd = day_avg_tide_pd.drop('gridY', 1)
day_avg_tide_pd = day_avg_tide_pd.drop('gridX', 1)
day_avg_tide_pd.to_csv('day_avg_tide_pd.csv')

In [ ]:
low_pass_tide = pd.rolling_mean(day_avg_tide_pd, 4, center=True)
low_pass_tide.to_csv('low_pass_tide.csv')
low_pass_tide.plot()